In [1]:
!pip install lime datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64"

In [ ]:
!git clone https://github.com/Efromomr/mamba

In [ ]:
%cd /content/mamba

In [ ]:
!pip install causal_conv1d --quiet
!pip install . --quiet

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.environ["TRITON_F32_DEFAULT"] = "ieee"

In [2]:
import string

from datasets import load_dataset
import numpy as np
import torch
from lime.lime_text import LimeTextExplainer
import transformers
from sklearn.metrics import precision_recall_curve
import gc, torch

from feedforward import SwiGLU
from model import LanguageModel
from mha import ModifiedMHA
from interpretation_methods import lime, my_attention
from evaluation import faithfulness

In [3]:
MODEL_NAME = 'efromomr/mamba_x2_attn'

In [4]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased', padding=True, truncation=True, )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
SwiGLU_cfg = {'dim': 512, 'dim_inner': 'null', 'drop_rate': 0, 'bias': False}
Mamba_cfg = {'dim': 512, 'd_state': 16*4, 'd_conv': 4, 'expand': 2, "headdim": 4, "A_init_range": (1, 1.1), 'compute_attn': True, "use_mem_eff_path": False}
Attn_cfg = {'embed_dim': 512, "num_heads": 128, 'batch_first': True}

In [6]:
check_model = LanguageModel.from_pretrained(MODEL_NAME,  layers = [Mamba2Simple, SwiGLU, Mamba2Simple, SwiGLU, ModifiedMHA, SwiGLU],
                      layer_cfgs = [Mamba_cfg, SwiGLU_cfg, Mamba_cfg, SwiGLU_cfg, Attn_cfg, SwiGLU_cfg], dim=512))
check_model = check_model.eval()

config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

In [7]:
dataset = load_dataset('eraser-benchmark/movie_rationales', trust_remote_code=True)

README.md:   0%|          | 0.00/6.65k [00:00<?, ?B/s]

movie_rationales.py:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/199 [00:00<?, ? examples/s]

In [8]:
lime_explainer = LimeTextExplainer(class_names = ['0', '1'])

In [12]:
import pickle

with open('movies.pickle', 'rb') as handle:
			dataset = pickle.load(handle)

In [13]:
text = []
labels = []
rationales=[]
for key in dataset.keys():
			text.append(dataset[key][0])
			if dataset[key][3]['classification'] == 'NEG':
				labels.append(0)
			else:
				labels.append(1)
			rationales.append(dataset[key][1])

In [14]:
test_test_rationales = []
for i in range(len(text)):
    if (labels[i] == 1):
        test_test_rationales.append([[0]*len(rationales[i]), rationales[i]])
    else:
        test_test_rationales.append([rationales[i],[0]* len(rationales[i])])

new_rationale= []
for i in range(len(text)):
    rationale = []
    test_t = text[i].split(' ')
    for j in range(2):
        label_rational = []
        for k in range(len(test_t)):
            for r in tokenizer(test_t[k]).input_ids[1:-1]:
                #if r == '.':
                #    print(r)
                #print(r)
                rationall = 1 if test_test_rationales[i][j][k] > 0 else 0
                label_rational.append(rationall)
        rationale.append(label_rational)
    new_rationale.append(rationale)

##Lime

In [ ]:
evaluation = {'faithfulness': faithfulness, 'truthfulness': truthfulness, 'auprc': auprc}

metrics = {'faithfulness': [], 'truthfulness': [], 'auprc': []}

for num, line in enumerate(text[:5]):
  interpretations = []

  input_ids = torch.tensor(tokenizer(line, max_length=512,padding='max_length', truncation=True).input_ids)[None, :]
  tokens = [tokenizer.decode(tok) for tok in tokenizer.encode(line, max_length=512,padding=False, truncation=True)]
  tokens = " ".join(tokens).replace(' ##', "").split(" ")

  prediction = check_model(input_ids)
  test_rational = new_rationale[num]

  temp = lime(line, _, tokens, _, _, _)
  interpretations.append([np.array(i)/np.max(abs(np.array(i))) for i in temp])

  for metric in ['faithfulness']:
            evaluated = []
            for interpretation in interpretations:
                evaluated.append(evaluation[metric](interpretation, _, line, prediction, tokens, _, _, test_rational))
            metrics[metric].append(evaluated)

In [26]:
np.mean(metrics['faithfulness'])

np.float64(0.026332974433898926)

##Attention-based

In [19]:
evaluation = {'faithfulness': faithfulness, 'truthfulness': truthfulness, 'auprc': auprc}

metrics = {'faithfulness': [], 'truthfulness': [], 'auprc': []}

attn_cfg = {'layers': 'Mean',
            'heads': 'Mean',
            'matrix': 'From',
            'selection': False}

for num, line in enumerate(text[:100]):
  interpretations = []

  input_ids = torch.tensor(tokenizer(line, max_length=512, padding=False, truncation=True).input_ids)[None, :]

  prediction = check_model(input_ids)

  modules_with_attn = []

  for name, module in check_model.named_modules():
      if isinstance(module, ModifiedMHA):# or isinstance(module, Mamba2Simple):
        modules_with_attn.append(module.attn_matrix)

  modules_with_attn = torch.cat(modules_with_attn)

  #test_rational = new_rationale[num]

  temp = my_attention(line, _, input_ids, _, modules_with_attn, _)
  interpretations.append([np.array(i)/np.max(abs(np.array(i))) for i in temp])

  for metric in ['faithfulness']:
            evaluated = []
            for interpretation in interpretations:
                evaluated.append(evaluation[metric](interpretation, _, line, prediction, input_ids, _, _, _, True))
            metrics[metric].append(evaluated[0][0].item() if evaluated[0][0] else evaluated[0][1].item())
  gc.collect()
  torch.cuda.empty_cache()
  del input_ids
  del modules_with_attn
  del temp
  del prediction
  del interpretations


In [56]:
np.mean(metrics['faithfulness'])

np.float64(0.027760279178619385)